In [1]:
from datetime import datetime
from elasticsearch import Elasticsearch
es = Elasticsearch([{'host': 'elasticsearch', 'port': 9200}])

/usr/local/lib/python2.7/dist-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.25.8) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [2]:
es.indices.get_alias("*")

{u'.apm-agent-configuration': {u'aliases': {}},
 u'.kibana_1': {u'aliases': {u'.kibana': {}}},
 u'.kibana_task_manager_1': {u'aliases': {u'.kibana_task_manager': {}}},
 u'montevideo': {u'aliases': {}}}

In [34]:
text = ' de Salto y (Bernabé) Rivera. '

In [35]:
results = es.search(index="montevideo", body=
{
    "query" : {
        "match" : { 
            "message" : {
                "query" : text,

            }
        }
    }
})
for result in results['hits']['hits']:
    print(result['_score'],result['_source']['type'],result['_source']['nombre'])

In [26]:
results = es.search(index="montevideo", body=
{
    "query" : {
        "query_string" : {
            "query" : "reconquista y perez"
        }
    }
})
for result in results['hits']['hits']:
    print(result['_score'],result['_source']['type'],result['_source']['nombre'])

(12.048357, u'v_mdg_vias', u'RECONQUISTA')
(12.048357, u'v_mdg_vias', u'RECONQUISTA')
(12.048357, u'v_mdg_vias', u'RECONQUISTA')
(6.7747774, u'v_mdg_vias', u'PSJE MANUEL PEREZ Y CURIS')
(6.7747774, u'v_mdg_vias', u'PSJE MANUEL PEREZ Y CURIS')
(6.1521435, u'geonames_uy_montevideo', u'Molino de Perez')
(6.0475097, u'v_mdg_vias', u'PEREZ CASTELLANO')
(6.0475097, u'v_mdg_vias', u'PEREZ CASTELLANO')
(6.0475097, u'v_mdg_vias', u'PABLO PEREZ')
(6.0475097, u'v_mdg_vias', u'PEREZ GOMAR')
